# Scraping Lyrics.lk using Scrapy



In [ ]:
!pip install Scrapy
!scrapy startproject lyrics
!scrapy crawl lyrics -o output.json

# Send documents to elasticsearch instance

In [ ]:
import json 
import requests

f = open('/lyrics/output.json',) 

data = json.load(f)   
f.close()

id = 0
header = {"Content-type": "application/json"} 

for i in data:
  r = requests.post("http://ec2-100-26-56-20.compute-1.amazonaws.com:9200/sinhala_songs/_doc/"+str(id), data=json.dumps(i), headers=header)
  print(r.status_code, r.reason)
  id+=1

# Scraping sinhalajukebox website using Selenium automation tool


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
def getArtists(url):

  my_artist_database = []

  wd.get(url)
  data = wd.find_elements_by_tag_name('a')
  start = 0
  for item in data:    
    text = item.text 
    data = text.split('\n')
    if len(data)>1:
      href = item.get_attribute('href')
      my_artist_database.append([text.split('\n')[0],href])
  
  return my_artist_database

In [ ]:
def scrape_artist(url):
  wd.get(url)
  data = wd.find_elements_by_tag_name('td')

  song_list = []
  song = []
  counter = 0
  ids = [1,3,4]
  start = False
  for item in data:    
    text = item.text 
    if text=="0001":
      start = True
    
    if start:
      if counter in ids:
        song.append(text)
      counter+=1

      try:
        link = item.find_element_by_tag_name('a')
        link = link.get_attribute('href')
        if "/lyrics/" in link:
          song.append(link)
          song_list.append(song)
          song = []
          counter=0
      except:
        continue
  
  return song_list

In [ ]:
def get_img(url):
  wd.get(url)
  data = wd.find_elements_by_tag_name('img')

  for d in data:
    link = d.get_attribute('src')
    if "/lyrics/" in link:
      return link

In [ ]:
categories = [["classical","http://sinhalajukebox.org/categories/C01.php/"] ,["baila","http://sinhalajukebox.org/categories/C04.php/"],["oldies","http://sinhalajukebox.org/categories/C05.php/"],["cultural","http://sinhalajukebox.org/categories/C08.php/"],["drama","http://sinhalajukebox.org/categories/C09.php/"],["instrumentals","http://sinhalajukebox.org/categories/C06.php/"],["new pop","http://sinhalajukebox.org/categories/C03.php/"],["various","http://sinhalajukebox.org/categories/C07.php/"] ,["english","http://sinhalajukebox.org/categories/C10.php/"],["movie","http://sinhalajukebox.org/categories/C11.php/"],["rap","http://sinhalajukebox.org/categories/C12.php/"]]

for category in categories:
  my_artist_database = getArtists(category[1])

  my_songs_database = []

  for i in my_artist_database:
    song = {}
    for j in scrape_artist(i[1]):
      song['singer'] = i[0]
      song['title'] = j[0]
      song['lyricist'] = j[1]
      song['tune composer'] = j[2]
      song['song'] = j[3]
      my_songs_database.append(song)
      song = {}
  
  for i in range(len(my_songs_database)):
    real_img_url = get_img(my_songs_database[i]['song'])
    my_songs_database[i]['song'] = real_img_url

  with open('category[0]+'.json', 'w') as f:
    json.dump(my_songs_database, f)

  print("saved ",category[0])

In [ ]:
f = open('baila.json',) 

data = json.load(f)   
f.close()

id = 0
header = {"Content-type": "application/json"} 

for i in data:
  r = requests.post("http://ec2-100-26-56-20.compute-1.amazonaws.com:9200/sinhala_songs/_doc/"+str(id), data=json.dumps(i), headers=header)
  print(r.status_code, r.reason)
  id+=1